In [8]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import pandas as pd
import psutil
import time
import sys

# Optional: Increase recursion limit (use with caution)
sys.setrecursionlimit(5000)

# Load 1% of the dataset
dataset = load_dataset("ag_news", split="train[:10%]")

# Load model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Split dataset into train and test (80% train, 20% test)
train_size = int(0.8 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
test_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

# Load metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # Convert to MB

# Function to fine-tune and evaluate the model
def fine_tune_and_evaluate(model, train_dataset, eval_dataset):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
    )
    
    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        labels = p.label_ids
        accuracy = accuracy_metric.compute(predictions=preds, references=labels)['accuracy']
        f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')['f1']
        return {"accuracy": accuracy, "f1": f1}
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    
    start_time = time.time()
    memory_before = get_memory_usage()
    
    trainer.train()
    
    training_time = time.time() - start_time
    results = trainer.evaluate()
    
    memory_after = get_memory_usage()
    memory_usage = memory_after - memory_before

    return {
        "accuracy": results["eval_accuracy"],
        "f1": results["eval_f1"],
        "training_time": training_time,
        "memory_usage": memory_usage,
    }

# Dictionary to store results
results_summary = {}

# Zero-Shot Learning Evaluation
zero_shot_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
zero_shot_results = fine_tune_and_evaluate(zero_shot_model, train_dataset, test_dataset)
results_summary["Zero-Shot (1% Data)"] = zero_shot_results

# Few-Shot Learning Evaluations with different shot sizes
for few_shot_size in [1, 5, 10, 50]:  # Adjust few-shot sizes as needed
    few_shot_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
    few_shot_train_dataset = train_dataset.shuffle(seed=42).select(range(few_shot_size))  # Select few-shot examples
    few_shot_results = fine_tune_and_evaluate(few_shot_model, few_shot_train_dataset, test_dataset)
    results_summary[f"Few-Shot ({few_shot_size})"] = few_shot_results

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 33%|███▎      | 120/360 [02:05<04:10,  1.04s/it]
                                                  

 14%|█▍        | 500/3600 [02:20<14:23,  3.59it/s]


{'loss': 0.4822, 'grad_norm': 0.4884715676307678, 'learning_rate': 4.305555555555556e-05, 'epoch': 0.42}


                                                   

 28%|██▊       | 1000/3600 [04:42<12:17,  3.53it/s]


{'loss': 0.3821, 'grad_norm': 8.839018821716309, 'learning_rate': 3.611111111111111e-05, 'epoch': 0.83}


                                                   

                                          
 33%|███▎      | 1200/3600 [06:03<10:57,  3.65it/s]


...

                                               
                                              

 67%|██████▋   | 14/21 [00:53<00:07,  1.11s/it] 


{'eval_loss': 1.2044686079025269, 'eval_accuracy': 0.6029166666666667, 'eval_f1': 0.5847078027954136, 'eval_runtime': 24.8127, 'eval_samples_per_second': 96.725, 'eval_steps_per_second': 12.091, 'epoch': 2.0}


                                               
                                              

100%|██████████| 21/21 [01:21<00:00,  1.18s/it] 
                                               

100%|██████████| 21/21 [01:21<00:00,  3.86s/it]


{'eval_loss': 1.1472645998001099, 'eval_accuracy': 0.6654166666666667, 'eval_f1': 0.661081531987111, 'eval_runtime': 24.8236, 'eval_samples_per_second': 96.682, 'eval_steps_per_second': 12.085, 'epoch': 3.0}
{'train_runtime': 81.1087, 'train_samples_per_second': 1.849, 'train_steps_per_second': 0.259, 'train_loss': 1.1700435820079984, 'epoch': 3.0}


100%|██████████| 300/300 [00:24<00:00, 12.16it/s]


In [15]:
# Function to evaluate the model without fine-tuning
def evaluate_model(model, eval_dataset):
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        logging_dir="./logs",  # Optional: specify logging directory
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=eval_dataset,
        compute_metrics=lambda p: {
            "accuracy": accuracy_metric.compute(predictions=p.predictions.argmax(-1), references=p.label_ids)['accuracy'],
            "f1": f1_metric.compute(predictions=p.predictions.argmax(-1), references=p.label_ids, average='weighted')['f1'],
        },
    )
    
    results = trainer.evaluate()
    return {
        "accuracy": results["eval_accuracy"],
        "f1": results["eval_f1"],
        "training_time": None,  # No training time for zero-shot
        "memory_usage": None,    # No memory usage for zero-shot
    }

# Load the model again for zero-shot evaluation
zero_shot_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Evaluate the model on the test dataset without fine-tuning
zero_shot_results = evaluate_model(zero_shot_model, test_dataset)
results_summary["Zero-Shot (After Initial Training)"] = zero_shot_results

# Print the results
print(results_summary)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




































































































































































































































































































































































































































































100%|██████████| 300/300 [00:24<00:00, 12.40it/s]

{'Zero-Shot (1% Data)': {'accuracy': 0.9229166666666667, 'f1': 0.9226888188503383, 'training_time': 1089.4382178783417, 'memory_usage': 161.046875}, 'Few-Shot (1)': {'accuracy': 0.27291666666666664, 'f1': 0.13745657182161553, 'training_time': 74.0447449684143, 'memory_usage': 7.234375}, 'Few-Shot (5)': {'accuracy': 0.32875, 'f1': 0.25914819892847807, 'training_time': 75.96170377731323, 'memory_usage': 10.109375}, 'Few-Shot (10)': {'accuracy': 0.32375, 'f1': 0.2164901052475208, 'training_time': 76.01416873931885, 'memory_usage': 18.375}, 'Few-Shot (50)': {'accuracy': 0.6654166666666667, 'f1': 0.661081531987111, 'training_time': 81.24914598464966, 'memory_usage': 7.640625}, 'Zero-Shot (After Initial Training)': {'accuracy': 0.17833333333333334, 'f1': 0.17282327816844673, 'training_time': None, 'memory_usage': None}}


In [16]:
results_summary

{'Zero-Shot (1% Data)': {'accuracy': 0.9229166666666667,
  'f1': 0.9226888188503383,
  'training_time': 1089.4382178783417,
  'memory_usage': 161.046875},
 'Few-Shot (1)': {'accuracy': 0.27291666666666664,
  'f1': 0.13745657182161553,
  'training_time': 74.0447449684143,
  'memory_usage': 7.234375},
 'Few-Shot (5)': {'accuracy': 0.32875,
  'f1': 0.25914819892847807,
  'training_time': 75.96170377731323,
  'memory_usage': 10.109375},
 'Few-Shot (10)': {'accuracy': 0.32375,
  'f1': 0.2164901052475208,
  'training_time': 76.01416873931885,
  'memory_usage': 18.375},
 'Few-Shot (50)': {'accuracy': 0.6654166666666667,
  'f1': 0.661081531987111,
  'training_time': 81.24914598464966,
  'memory_usage': 7.640625},
 'Zero-Shot (After Initial Training)': {'accuracy': 0.17833333333333334,
  'f1': 0.17282327816844673,
  'training_time': None,
  'memory_usage': None}}

In [17]:
# Create a DataFrame from the results_summary
results_df = pd.DataFrame.from_dict(results_summary, orient='index')

# Reset index to have Method as a column
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'Method'}, inplace=True)

# Export the DataFrame to a CSV file
results_df.to_csv("results_summary.csv", index=False)

# Print the DataFrame to verify
print(results_df)

                               Method  accuracy        f1  training_time  \
0                 Zero-Shot (1% Data)  0.922917  0.922689    1089.438218   
1                        Few-Shot (1)  0.272917  0.137457      74.044745   
2                        Few-Shot (5)  0.328750  0.259148      75.961704   
3                       Few-Shot (10)  0.323750  0.216490      76.014169   
4                       Few-Shot (50)  0.665417  0.661082      81.249146   
5  Zero-Shot (After Initial Training)  0.178333  0.172823            NaN   

   memory_usage  
0    161.046875  
1      7.234375  
2     10.109375  
3     18.375000  
4      7.640625  
5           NaN  
